In [24]:
# importing libraries
import pandas as pd
import numpy as np
from collections import Counter

In [28]:
# reading the features of the training dataset
x_train = pd.read_csv('example_data/train-features.txt', sep = " ", header = None)
x_train = x_train.to_numpy()
# reading the label of the training dataset
y_train = pd.read_csv('example_data/train-labels.txt', sep = '\n', header = None)
y_train = y_train[0].values.tolist()
# reading the features of the test dataset
x_test = pd.read_csv('example_data/test-features.txt', sep = " ", header = None)
x_test = x_test.to_numpy()
# reading the labels of the test dataset
y_test = pd.read_csv('example_data/test-labels.txt', sep = '\n', header = None)
y_test = y_test[0].values.tolist()

# checking dimensions of the dataset
print("dimensions of the training features dataset:", x_train.shape)
print("no. of emails in the training features = no. of emails in the training labels?", len(y_train) == x_train.shape[0])
print("dimensions of the test features dataset:", x_test.shape)
print("no. of emails in the test features = no. of emails in the test labels?", len(y_test) == x_test.shape[0])

# checking the number of classes in the dataset and whether they are imbalanced
print("classes in the training dataset and their counts:", dict(Counter(y_train)))
print("classes in the training dataset and their counts:", dict(Counter(y_test)))

dimensions of the training features dataset: (700, 2500)
no. of emails in the training features = no. of emails in the training labels? True
dimensions of the test features dataset: (260, 2500)
no. of emails in the test features = no. of emails in the test labels? True
classes in the training dataset and their counts: {0: 350, 1: 350}
classes in the training dataset and their counts: {0: 130, 1: 130}
